<a href="https://colab.research.google.com/github/ssunwalka01/NER/blob/master/Slintel_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- importing the libraries here...

In [337]:
import nltk
# nltk.download()
import pandas as pd
import spacy
# from spacy import displacy
# from collections import Counter
import en_core_web_lg
nlp = en_core_web_lg.load()

In [338]:
# !python -m spacy download en_core_web_lg

In [339]:
master = pd.read_csv("/content/master-company-sheet.csv")
blogs = pd.read_csv("/content/news-data.csv")
output = pd.read_csv("/content/output-sheet.csv")

In [340]:
print("master sheet", master.head())
print("\n\nmaster sheet data types\n", master.dtypes)

master sheet    _id                            name
0    1                      Telefónica
1    2             Corsair Capital LLC
2    3                            MSTS
3    4  Franklin Templeton Investments
4    5                      Legg Mason


master sheet data types
 _id      int64
name    object
dtype: object


In [341]:
print("blogs sheet", blogs.head())
print("\n\n blogs sheet data types\n", blogs.dtypes)

blogs sheet    _id  ...                  published_on
0    1  ...  2020-07-31T15:41:00.0000000Z
1    2  ...  2020-08-01T00:02:00.0000000Z
2    3  ...  2020-07-31T22:36:00.0000000Z
3    4  ...  2020-08-01T06:29:00.0000000Z
4    5  ...  2020-07-31T18:47:00.0000000Z

[5 rows x 6 columns]


 blogs sheet data types
 _id              int64
theme           object
source          object
title           object
description     object
published_on    object
dtype: object


In [342]:
print("output sheet", output.head())
print("\n\n output sheet data types\n", output.dtypes)

output sheet Empty DataFrame
Columns: [blog_id, title, description, company_ids]
Index: []


 output sheet data types
 blog_id        object
title          object
description    object
company_ids    object
dtype: object


In [343]:
def dictionary_with_title_description(df):
  " This function will return dictionaries containing blog id, company names from title, and company name from description"
  
  company_names_title = {} # initialising empty dictionary for company names extracted from title
  company_names_description = {} # initialising empty dictionary for company names extracted from description
  descriptionDict = {} # {index : [blog_id, title, description], index_2 : ...}
  
  for i in range(0, len(df)):
    
    descriptionDict.setdefault(i, [])
    # The magic of setdefault is that it initializes the value for that key if that key is not defined, otherwise it does nothing.
    descriptionDict[i].append(df.loc[i, "_id"]) # will be adding multiple values in a single key (Here, adding _id)

    #  title
    descriptionDict.setdefault(i, [])
    descriptionDict[i].append(df.loc[i, "title"]) # adding title as value in dictionay

    # description
    descriptionDict.setdefault(i, [])
    descriptionDict[i].append(df.loc[i, "description"]) #adding description as value in the dictionary

  for i in range(0, len(df)):
    doc = nlp(descriptionDict[i][2])

    for X in doc.ents:
      #print((X.text, X.label_, type(X.text), type(X.label_)))
      
      if X.label_ == 'ORG':
        company_names_description.update({descriptionDict[i][0]:X.text})

  for i in range(0, len(df)):
    titl = nlp(descriptionDict[i][1])
    
    for X in titl.ents:
      #print((X.text, X.label_, type(X.text), type(X.label_)))
      
      if X.label_ == 'ORG':
        company_names_title.update({descriptionDict[i][0]:X.text})

  return descriptionDict, company_names_description, company_names_title

In [344]:
descriptionDict, company_names_description, company_names_title = dictionary_with_title_description(blogs)

In [345]:
final_dictionary = {}
try:
  for key, value in company_names_description.items():
    # print("Value at o", value[0])
    
    for j in range(0, len(master)):
      
      if (value == master.loc[j, "name"]):
        
        final_dictionary.setdefault(key, [])
        final_dictionary[key].append(master.loc[j, "_id"])

except:
  print("RA")

In [346]:
final_dictionary

{6: [36], 7: [38], 13: [42], 25: [45], 78: [29], 107: [22]}

In [347]:
try:
  for key, value in company_names_title.items():
    # print("Value at o", value[0])
    
    for j in range(0, len(master)):
      
      if (value == master.loc[j, "name"]):
        # print("name", value,"---", master.loc[j, "name"])
        final_dictionary.setdefault(key, [])
        final_dictionary[key].append(master.loc[j, "_id"])


except:
  print("RA")

In [348]:
blogs.set_index("_id")

,theme,source,title,description,published_on
_id,,,,,
1,funding,bing,Chinese Tesla rival Xpeng Motors in talks over...,"Xpeng Motors, one of Tesla's rivals in China, ...",2020-07-31T15:41:00.0000000Z
2,funding,bing,Wine.com Seeks Funding at Valuation of More Th...,"Wine.com, an online retailer that lets custome...",2020-08-01T00:02:00.0000000Z
3,funding,bing,Fenwick-Led Telehealth Co. Raises $33M As Indu...,Fenwick-advised telehealth platform Lemonaid H...,2020-07-31T22:36:00.0000000Z
4,funding,bing,United Community Banks (NASDAQ:UCBI) Raised to...,United Community Banks (NASDAQ:UCBI) was upgra...,2020-08-01T06:29:00.0000000Z
5,funding,bing,"Golden West Food Group, Cooks Venture Raise $1...",PRNewswire/ -- Golden West Food Group and Cook...,2020-07-31T18:47:00.0000000Z
...,...,...,...,...,...
109,ipos,bing,AlloVir raises $276M IPO to run broad cell the...,AlloVir has raised $276 million in an upsized ...,2020-07-31T12:25:00.0000000Z
110,ipos,bing,Exclusive: SenseTime eyes STAR market IPO afte...,Chinese start-up SenseTime is considering an i...,2020-07-31T10:48:00.0000000Z
111,ipos,bing,Vital Farms shares soar 63% in trading debut a...,Shares of ethical food company Vital Farms Inc...,2020-07-31T16:45:00.0000000Z


In [349]:
for k, v in final_dictionary.items():
  new_row = {'blog_id':k, 'title': blogs["title"][k], 'description': blogs["description"][k], 'company_ids': final_dictionary[k]}
  output = output.append(new_row, ignore_index=True)

In [350]:
output

,blog_id,title,description,company_ids
0,6,"XBiotech: Well Funded, Working On The Next Ber...",XBiotech develops True Humanâ„¢ (naturally occ...,"[36, 36]"
1,7,Kodak boss took out stock options day before f...,"Kodak CEO, Jim Continenza (above) was awarded ...",[38]
2,13,"Microsoft in talks to acquire TikTok, reports say",Microsoft is reportedly in talks to acquire Ti...,"[42, 42]"
3,25,Microsoft Reportedly In Talks To Acquire TikTo...,We may be banning TikTok. We are looking at a ...,"[45, 45]"
4,78,RPN launches improved Rubber News Directory in...,"In conjunction with the launch, MediaBrains is...","[29, 29]"
5,107,Eventbrite co-founder joins 'blank check' IPO ...,"Kevin Hartz, co-founder of Eventbrite Inc., fi...",[22]
6,1,Wine.com Seeks Funding at Valuation of More Th...,"Wine.com, an online retailer that lets custome...",[31]
7,8,Mammoth Energy Services' (TUSK) CEO Arty Strae...,"Q2 2020 Earnings Conference Call July 30, 2020...",[40]
8,18,Alta Equipment to acquire Martin Implement Sales,Alta Equipment Group (NYSE:ALTG) announcesÂ an...,[43]
9,19,Advent to acquire RA Chem Pharma,Advent International has agreed to acquire a c...,[43]


In [351]:
output.to_csv("output-sheet_new_lg.csv", index = False)